In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns

match_info = pd.read_csv("data/matches_info.csv")
possessions = pd.read_csv("data/possessions.csv")
run_features = pd.read_csv("data/run_features.csv")
tracking_data = pd.read_csv("data/tracking_data.csv")
player_to_team = pd.read_csv("data/player_team.csv").set_index("id")

merged = pd.merge(possessions,run_features,on=["match_id","possession_index"],how="outer",suffixes=("_possession","_run"))

merged["possession_lead_to_shot"] = (merged["possession_lead_to_shot"] | merged["lead_to_shot"])# Need to update wheter runs and possessions lead to shots on values that conflict
merged["possession_lead_to_goal"] = (merged["possession_lead_to_goal"] | merged["lead_to_goal"])


In [146]:
run_types_to_predict = ['run_ahead_of_the_ball','cross_receiver','behind']
run_to_predict = merged[merged.event_subtype.isin(run_types_to_predict)]

# Build Graph

In [12]:
import torch
from torch.utils.data import Dataset
from torch_geometric.data import Data
import numpy as np

def build_graph_from_frame(
    frame_df,
    runner_player_id,
    player_to_team,
    R=15.0
):
    """
    frame_df: rows = players at ONE frame_id
    """

    # -------------------------------------------------
    # 1. Sort & extract players
    # -------------------------------------------------

    frame_df = frame_df.sort_values("player").reset_index(drop=True)

    player_ids = frame_df["player"].values
    Np = len(player_ids)

    # ---------------- Player node features ----------------

    player_feats = torch.tensor(
        frame_df[[
            "x", "y",
            "dx_smooth", "dy_smooth",
            "speed",
            "acceleration"
        ]].values,
        dtype=torch.float
    )

    team_ids = torch.tensor(
        player_to_team.loc[player_ids, "team_id"].values,
        dtype=torch.long
    )

    # runner flag for players
    is_runner_player = torch.tensor(
        (player_ids == runner_player_id).astype(float),
        dtype=torch.float
    ).unsqueeze(1)

    # append 0 for ball
    is_runner = torch.cat(
        [is_runner_player, torch.zeros(1, 1)],
        dim=0
    )

    is_ball = torch.zeros(Np + 1, dtype=torch.bool)

    # -------------------------------------------------
    # 2. Ball node
    # -------------------------------------------------

    ball_feats = torch.tensor(
        [[
            frame_df["ball_x"].iloc[0],
            frame_df["ball_y"].iloc[0],
            frame_df["ball_dx_smooth"].iloc[0],
            frame_df["ball_dy_smooth"].iloc[0],
            frame_df["ball_speed"].iloc[0],
            frame_df["ball_acceleration"].iloc[0],
        ]],
        dtype=torch.float
    )

    # -------------------------------------------------
    # 3. Combine nodes
    # -------------------------------------------------

    x = torch.cat([
        player_feats,
        ball_feats
    ], dim=0)

    # add indicator features
    is_ball[-1] = True
    is_ball_feat = is_ball.float().unsqueeze(1)

    team_ids = torch.cat([
        team_ids,
        torch.tensor([-1])
    ])

    x = torch.cat([
        x,
        is_runner,
        is_ball_feat
    ], dim=1)

    # -------------------------------------------------
    # 4. Build edges
    # -------------------------------------------------

    pos = x[:, :2]
    vel = x[:, 2:4]
    N = pos.size(0)

    edge_index = []
    edge_attr = []

    for i in range(N):
        for j in range(N):
            if i == j:
                continue

            dist = torch.norm(pos[i] - pos[j])

            if (dist < R) or is_ball[i] or is_ball[j]:

                same_team = int(
                    (team_ids[i] == team_ids[j]) and
                    (team_ids[i] != -1)
                )

                ball_edge = int(is_ball[i] or is_ball[j])

                # relative velocity projected onto edge
                rel_v = vel[j] - vel[i]
                direction = (pos[j] - pos[i]) / (dist + 1e-6)
                rel_speed = torch.dot(rel_v, direction)

                edge_index.append([i, j])
                edge_attr.append([
                    dist.item(),
                    same_team,
                    ball_edge,
                    rel_speed.item()
                ])

    edge_index = torch.tensor(edge_index, dtype=torch.long).T
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # -------------------------------------------------
    # 5. Runner index
    # -------------------------------------------------

    runner_idx = int(
        torch.where(
            torch.tensor(player_ids) == runner_player_id
        )[0].item()
    )

    # -------------------------------------------------
    # 6. PyG Data object
    # -------------------------------------------------

    return Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        runner_idx=torch.tensor(runner_idx, dtype=torch.long),
        team_ids=team_ids,
        is_ball=is_ball
    )
    

In [ ]:
RUN_NUM = 0
RUN_FRAME = 2
run = run_to_predict.iloc[RUN_NUM]
match_id = run["match_id"]
run_id = run["id"]
player = run["player_id"]

run_tracking = tracking_data[(tracking_data.run_id == run_id) & (tracking_data.match_id == match_id)]
run_tracking["timestamp"] = pd.to_timedelta(run_tracking["timestamp"]).dt.total_seconds()
frame_num = run_tracking["frame_id"].sort_values().iloc[0] + RUN_FRAME
frame_df = run_tracking[run_tracking["frame_id"]==frame_num]


build_graph_from_frame(frame_df=frame_df,runner_player_id=player,player_to_team=player_to_team)

C:\Users\zcoch\AppData\Local\Temp\ipykernel_2372\1346443168.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_tracking["timestamp"] = pd.to_timedelta(run_tracking["timestamp"]).dt.total_seconds()


Data(x=[23, 8], edge_index=[2, 130], edge_attr=[130, 4], runner_idx=1, team_ids=[23], is_ball=[23])

# Temporal Dataset class

In [162]:
from torch_geometric.data import Batch

class TemporalRunnerDataset(Dataset):
    def __init__(self, tracking_df, run_features, player_to_team, samples, T=10, horizon=1, R=15.0):
        self.df = tracking_df
        self.run_features = run_features
        self.player_to_team = player_to_team
        self.samples = samples
        self.T = T
        self.horizon = horizon
        self.R = R

        # group for fast lookup
        self.df = self.df.sort_values(["match_id", "frame_id"])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        match_id,run_id, frame_t, runner_id = self.samples[idx]

        graphs = []

        # -------- temporal graphs --------
        for t in range(frame_t - self.T + 1, frame_t + 1):

            frame_df = self.df[
                (self.df["match_id"] == match_id) & 
                (self.df["run_id"]==run_id) &
                (self.df["frame_id"] == t)
            ]

            g = build_graph_from_frame(
                frame_df=frame_df,
                runner_player_id=runner_id,
                player_to_team=self.player_to_team,
                R=self.R
            )

            graphs.append(g)
            
        graphs_batch = Batch.from_data_list(graphs)

        # -------- target: future runner velocity --------
        future = self.df[
            (self.df["match_id"] == match_id) & 
            (self.df["run_id"]==run_id) &
            (self.df["frame_id"] == frame_t + self.horizon) &
            (self.df["player"] == runner_id)
        ]
        
        target = torch.tensor(
            future[["dx_smooth", "dy_smooth"]].values.squeeze(),
            dtype=torch.float
        )
        
        lead_to_shot_val = self.run_features.loc[self.run_features['id'] == run_id, 'possession_lead_to_shot'].values
        if len(lead_to_shot_val) == 0:
            shot_label = torch.tensor(0.0)  # or handle missing
        else:
            shot_label = torch.tensor(float(lead_to_shot_val[0]))

        return graphs_batch, target, shot_label

def build_samples_from_runs(run_features, T, horizon):
    samples = []

    for _, run in run_features.iterrows():
        match_id = run["match_id"]
        run_id = run["event_id"]
        runner_id = run["player_id"]

        frame_start = int(run["frame_start_run"])
        frame_end = int(run["frame_end_run"])

        # earliest frame we can use (need T past frames)
        t_min = frame_start - T - 1
        t_max = frame_end - horizon

        if t_min > t_max:
            continue  # run too short

        for t in range(t_min, t_max + 1):
            samples.append(
                (match_id,run_id, t, runner_id)
            )

    return samples

In [163]:
T = 10
horizon = 1

samples = build_samples_from_runs(run_to_predict, T, horizon)

In [164]:
dataset = TemporalRunnerDataset(
    tracking_df=tracking_data,
    run_features=run_to_predict,
    player_to_team=player_to_team,
    samples=samples,
    T=10,
    horizon=1,
    R=15.0
)
dataset

# Temporal GNN

In [165]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv  # or other GNN layer

class TemporalRunnerGNN(nn.Module):
    def __init__(self, node_feat_dim, gnn_hidden_dim, rnn_hidden_dim):
        super().__init__()

        self.gnn1 = GCNConv(node_feat_dim, gnn_hidden_dim)
        self.gnn2 = GCNConv(gnn_hidden_dim, gnn_hidden_dim)

        self.rnn = nn.GRU(
            input_size=gnn_hidden_dim,
            hidden_size=rnn_hidden_dim,
            batch_first=True
        )

        self.pos_head = nn.Linear(rnn_hidden_dim, 2)
        self.shot_head = nn.Linear(rnn_hidden_dim, 1)

    def forward(self, graphs_batch):
        """
        graphs_batch: PyG Batch containing T graphs (time steps)
        """
        device = next(self.parameters()).device
        graphs_batch = graphs_batch.to(device)

        x, edge_index = graphs_batch.x, graphs_batch.edge_index

        # ---- GNN ----
        x = F.relu(self.gnn1(x, edge_index))
        x = F.relu(self.gnn2(x, edge_index))

        # ---- Extract runner embeddings per timestep ----
        batch_vec = graphs_batch.batch          # [num_nodes]
        runner_idx = graphs_batch.runner_idx    # [T]

        T = runner_idx.size(0)
        runner_embeds = []

        for t in range(T):
            node_mask = (batch_vec == t)
            node_indices = node_mask.nonzero(as_tuple=False).view(-1)

            global_runner_idx = node_indices[runner_idx[t]]
            runner_embeds.append(x[global_runner_idx])

        # [T, H] → [1, T, H]
        runner_embeds = torch.stack(runner_embeds).unsqueeze(0)

        # ---- Temporal model ----
        out, _ = self.rnn(runner_embeds)
        last_out = out[:, -1, :]  # [1, H]

        pred_pos = self.pos_head(last_out).squeeze(0)        # [2]
        pred_shot = self.shot_head(last_out).squeeze(0)      # [1]

        return pred_pos, pred_shot


# Training Loop

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch_geometric.loader import DataLoader as GeoDataLoader
from tqdm import tqdm

# Hyperparams
alpha = 1.0  # weight for shot loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume model, dataset are already created
model = TemporalRunnerGNN(node_feat_dim=8, gnn_hidden_dim=64, rnn_hidden_dim=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# dataloader = DataLoader(dataset, batch_size=1, shuffle=True)  # batch_size=1 for now
dataloader = GeoDataLoader(dataset, batch_size=1, shuffle=True)
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    total_loss = 0

    for graphs, target, shot_label in tqdm(dataloader):
        target = target.to(device).squeeze(0)
        shot_label = shot_label.to(device)

        optimizer.zero_grad()

        pred_pos, pred_shot = model(graphs)
        
        assert pred_pos.shape == target.shape
        assert pred_shot.shape == shot_label.shape
        
        weight = shot_label + 0.1 
        
        loss_pos = weight * F.mse_loss(pred_pos, target)
        loss_shot = F.binary_cross_entropy_with_logits(pred_shot.view(1),shot_label.view(1))
        loss = loss_pos + alpha * loss_shot
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: {total_loss / len(dataloader):.4f}")

  0%|          | 0/75770 [00:00<?, ?it/s]

  0%|          | 54/75770 [05:48<105:00:57,  4.99s/it]